<a href="https://colab.research.google.com/github/codereyinish/OpenSourceMBGPT/blob/main/QLora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
!pip uninstall torch torchvision -y
!pip install torch==2.1.0 torchvision==0.16.0


Found existing installation: torch 2.3.1+cu121
Uninstalling torch-2.3.1+cu121:
  Successfully uninstalled torch-2.3.1+cu121
Found existing installation: torchvision 0.18.1+cu121
Uninstalling torchvision-0.18.1+cu121:
  Successfully uninstalled torchvision-0.18.1+cu121
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 8.5 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.3.1
    Uninstalling triton-2.3.1:
      Successfully uninstalled triton-2.3.1
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.20.5
    Uninstalling nvidia-nccl-cu12-2.20.5:
      Successfully uninstalled nvidia-nccl-cu12-2.20.5
ERROR: pip's dependency resolver does not currently take into account

In [ ]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, DataCollatorForLanguageModeling
!pip install peft datasets
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

LOAD MODEL

In [ ]:
#Load the quantized model from hugging face transformer's library
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="main") # which version of model to use in repo

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4565: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.la

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

LOAD TOKENIZER

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

INFERENCE

In [ ]:
comment = "Great content, thank you!"
prompt=f'''[INST] {comment} [/INST]'''

In [ ]:
#Tokenize the text into numerical form, feed it inside the model and generate result and deco de it with the same tokenizer which we used to encode
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
#Extracting input_ids and attention_mask from the inputs
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']
print(attention_mask)
output = model.generate(inputs = input_ids.cuda() , temperature = 0.7, do_sample = True, top_p = 0.95, top_k = 40 , max_new_tokens = 512, attention_mask = attention_mask.to("cuda"), pad_token_id = tokenizer.eos_token_id)
print(tokenizer.decode(output[0]))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
<s> [INST] Great content, thank you! [/INST] I'm glad you found the content helpful! If you have any specific questions or topics you'd like me to cover in the future, please don't hesitate to ask. I'm here to help.

Regarding your feedback on the content, I'm always looking for ways to improve. Do you have any suggestions for how I could make the information more clear, engaging, or useful for my readers? I'd be happy to hear your thoughts.

Additionally, if you found the content valuable and would like to support my work, you can do so by sharing it with others, leaving a comment, or becoming a patron on my Patreon page. Your support helps me continue creating valuable content and resources for people like you.

Thank you again for your kind words and feedback! I appreciate it more than you know.</s>


`Well the response is long but after lots of mistkes, I am glad we got at least decent respone with redundancies which we will figure out with fine-tuning Later.

## PROMPT ENGINEERING

In [ ]:
intstructions_string = f"""MBGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–MBGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Please respond to the following comment.
"""

#prompt_template is a reference to Anonymous LAMBDA Function --> that generate a Prompt(special tokens(beginning) + instruction string + original prompt + special token(ending)) in standard format
prompt_template = lambda comment : f''' [INST] {intstructions_string} \n {comment} \n [\INST] '''
prompt = prompt_template(comment)
print(prompt)


 [INST] MBGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and ends responses with its signature '–MBGPT'. ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Please respond to the following comment.
 
 Great content, thank you! 
 [\INST] 


In [ ]:
tokenizer.pad_token = tokenizer.eos_token
#tokenize input
inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)
attention_mask = inputs['attention_mask']
#generate output
output = model.generate(inputs= inputs['input_ids'].cuda(), temperature= 0.7, do_sample = True, top_p = 0.95, top_k = 40 , max_new_tokens = 200,attention_mask = attention_mask.to("cuda"), pad_token_id = tokenizer.pad_token_id)
print(tokenizer.decode(output[0],skip_special_tokens=True))

 [INST] MBGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. It reacts to feedback aptly and ends responses with its signature '–MBGPT'. ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, thus keeping the interaction natural and engaging.

Please respond to the following comment.
 
 Great content, thank you! 
 [\INST] –MBGPT: Thank you for your kind words! I'm glad you found the content helpful. If you have any specific questions or topics you'd like me to cover, feel free to ask! –MBGPT.


Well, after little bit of Prompt Enginnering , we got decent  short and sweet response

MODEL configuration for Memory and Speed Jugaad...Plus training in right way for better eval accuracy

In [ ]:
model.train() #here dropout and batch normalization are implemented . Dropout to prevent overfitting/memorization or enhance generalizationa and Batch Normalization to make training data in same range

model.gradient_checkpointing_enable() #Does recomputation on Activations rather than  savings them in each small checkpoints

model = prepare_model_for_kbit_training(model)#training on less memory bit of training data --> easy and fast computation



In [ ]:
#Applying LoRA for training only few specific parameter instead of whole
peft_Config = LoraConfig(task_type ="CAUSAL_LM",  #for casual task --> text generation
                         r=8,
                         lora_alpha = 32,
                         lora_dropout = 0.05,
                         target_modules = ["q_proj"],
                         bias = "none"

)
model = get_peft_model(model, peft_Config) #This new model has most portion of the parameters freezed
#trainable parameter count
model.print_trainable_parameters()


trainable params: 2,097,152 || all params: 264,507,392 || trainable%: 0.7929


**PREPARE TRAINING DATASET.**

In [ ]:
#load dataset
data = load_dataset("shawhin/shawgpt-youtube-comments")


Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9 [00:00<?, ? examples/s]

In [ ]:
def tokenize_function(examples):
  text = examples["example"]
  tokenizer.truncation_side = "left"
  #tokenize
  tokenized_inputs = tokenizer(
      text,
      return_tensors = "np",
      max_length = 512,
      truncation = True)
  return tokenized_inputs

#Add padding
if tokenizer.pad_token is None:
  tokenizer.pad_token = tokenizer.eos_token
#update embedding length
model.resize_token_embeddings(len(tokenizer))


Embedding(32000, 4096, padding_idx=0)

In [ ]:
#tokenize both datasets(training and validation datasets)
tokenized_data = data.map(tokenize_function, batched = True)
#apply data collator between sequences
data_collator = DataCollatorForLanguageModeling(tokenizer,mlm=False)

### Fine Tuning

In [ ]:
#Setting hyperparameters
lr = 2e-4
batch_size = 4
num_epochs = 10

#defining training_arguments
training_args = transformers.TrainingArguments(
    output_dir = "MBGPT-ft",
    learning_rate = lr,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = num_epochs,
    weight_decay = 0.01,
    logging_strategy = "epoch",
    save_strategy = "epoch",
    evaluation_strategy = "epoch",
    load_best_model_at_end = True,
    gradient_accumulation_steps = 4, #mimick large batch size training
    warmup_steps = 2,
    fp16 = True,
    optim = "paged_adamw_8bit"

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = transformers.Trainer(
    model = model,
    args = training_args, #hyperparameter to set the model in right training mode
    train_dataset = tokenized_data["train"], #dataset to train the model
    eval_dataset = tokenized_data["test"], #dataset to validate the model
    data_collator = data_collator #for padding over batches
)

#train model
model.config.use_cache= False #Caching while training is memory intensive so better not use it
trainer.train()
#Enable Cache for Faster Causal LLM Inference
model.config.use_cache = True

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,4.592900,3.961789
1,4.041800,3.434592
2,3.461400,2.980977
4,2.666000,2.308795
5,2.357700,2.124309
6,2.178700,2.019431
8,2.003700,1.905994
9,1.390500,1.893574


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

YES YES🙌 < LETS PUSH THE MODEL TO THE HUB

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
hf_name = "HugManish22"
model_id = hf_name + "/" + "mbgpt-ft"

In [ ]:
model.push_to_hub(model_id) #push model to hub
trainer.push_to_hub(model_id)#push training artifacts like the training arguments, optimizer states, and potentially logs for reproducility of model


adapter_model.safetensors:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

events.out.tfevents.1723345442.96faa7c685d0.570.0:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/HugManish22/mbgpt-ft/commit/d84b35d68813dc41cbca34633b0a3f93b71480c5', commit_message='HugManish22/mbgpt-ft', commit_description='', oid='d84b35d68813dc41cbca34633b0a3f93b71480c5', pr_url=None, pr_revision=None, pr_num=None)